In [1]:
import pandas as pd
import numpy as np
import en_core_web_sm
from multiprocessing import Pool#multi-threads
pool = Pool(10)
nlp = en_core_web_sm.load()

In [2]:
from data_reader_general import data_reader, data_generator
class config:
    is_stanford_nlp = False
    embed_path = 'data/tweets/vocab/lower_local_emb.pkl'
    word_num = 19174
    embed_dim = 300
    char_num = 150
    char_dim = 128
    l_hidden_size =128
    mask_dim = 30
    dic_path = 'data/tweets/vocab/lower_dict.pkl'
    pretrained_embed_path = '../data/glove.840B.300d.txt'
    train_path = 'data/tweets/train_lower.pkl'
    valid_path = 'data/tweets/valid_lower.pkl'
    test_path = 'data/tweets/test_lower.pkl'
    data_path = 'data/tweets/'
    elmo_config_file = None
    elmo_weight_file= None
    use_label_dict = False
    batch_size = 8
    dropout = 0.2
    if_gpu = False
dr = data_reader(config)

In [3]:
train_data = dr.load_data(config.train_path)
valid_data = dr.load_data(config.valid_path)
test_data = dr.load_data(config.test_path)

In [4]:
print("Training Samples: {}".format(len(train_data)))
print("Validating Samples: {}".format(len(valid_data)))
print("Testing Samples: {}".format(len(test_data)))

Training Samples: 5206
Validating Samples: 1042
Testing Samples: 692


In [5]:
dg_train = data_generator(config,train_data)

In [6]:
sent_ids, mask_vecs, label_list, sent_lens, batch_char_ids_tensor, batch_char_lens_tensor, tokens = next(dg_train.get_char_ids_samples())

[INFO/ForkPoolWorker-11] child process calling self.run()
[INFO/ForkPoolWorker-12] child process calling self.run()
[INFO/ForkPoolWorker-13] child process calling self.run()
[INFO/ForkPoolWorker-14] child process calling self.run()
[INFO/ForkPoolWorker-15] child process calling self.run()
[INFO/ForkPoolWorker-16] child process calling self.run()
[INFO/ForkPoolWorker-17] child process calling self.run()
[INFO/ForkPoolWorker-18] child process calling self.run()
[INFO/ForkPoolWorker-19] child process calling self.run()
[INFO/ForkPoolWorker-20] child process calling self.run()


In [7]:
from hier_layer import SimpleCat

In [8]:
layer = SimpleCat(config)

In [9]:
sent_vec, char_output = layer(sent_ids, mask_vecs, sent_lens, batch_char_ids_tensor, batch_char_lens_tensor)

In [10]:
char_output.size()

torch.Size([8, 32, 158])

In [11]:
sent_vec.size()

torch.Size([8, 32, 330])

In [33]:
char_output.size()

torch.Size([8, 33, 300])

In [13]:
batch_char_lens_tensor[1]

tensor([ 2,  1,  7,  1,  1,  2,  3,  4,  2,  6,  3, 10,  3,  3,  2,  3,  6,  3,
         3,  2,  5,  5,  1,  4,  7,  2,  3,  1,  0,  0,  0,  0,  0])

In [10]:
sent_lens[0]

tensor(33)

Calculate the number of unique characters

In [26]:
text = [item[-3] for item in train_data]
text += [item[-3] for item in valid_data]
text += [item[-3] for item in test_data]

In [27]:
from collections import Counter
long_text = ' '.join(text)

In [28]:
char_freq = Counter(long_text).most_common(138)
len(char_freq)

138

In [30]:
chars, _ = zip(*char_freq)
#chars = chars[1:]

In [32]:
import pickle
char_id = {char:i for i, char in enumerate(chars)}
id_char = {i:char for i, char in enumerate(chars)}
with open('data/tweets/vocab/char_dict', 'wb') as f:
    pickle.dump([char_id, id_char, chars], f)

In [ ]:
batch_char_lens_tensor